In [2]:
from transformers import RobertaTokenizer, RobertaModel
import torch
import sys
sys.path.append('./src')
import syllo_gen
import random
import json
from tqdm.autonotebook import tqdm
import data
import syllo_finetune

%load_ext autoreload
%autoreload 2



In [3]:
pretrain_data = []
for t in ['adj', 'noun']:
    for i in range(6):
        pretrain_data.append(data.SYLLO(t, num_samples=(i+1)*1000, depth=i+1))
combine = torch.utils.data.ConcatDataset(pretrain_data)

In [4]:
len(combine)

42000

In [ ]:
folio_tr = data.FOLIO()
folio_te = data.FOLIO(split='dev')
folio_tr[2]

In [ ]:
model1 = syllo_finetune.LMCLS('roberta-large')
model = syllo_finetune.LMFOLIO('roberta-large', model1.roberta)


In [ ]:
model = syllo_finetune.LMFOLIO('roberta-large')
tokenizer = RobertaTokenizer.from_pretrained('roberta-large')
train_loader = torch.utils.data.DataLoader(folio_tr, batch_size=4, shuffle=True, collate_fn=syllo_finetune.collate_fn(tokenizer, False))
test_loader = torch.utils.data.DataLoader(folio_te, batch_size=16, shuffle=True, collate_fn=syllo_finetune.collate_fn(tokenizer, False))

In [ ]:
syllo_finetune.train(model, train_loader, test_loader=test_loader, epoch=2, fp16=True, lr=2e-5, warmup=0.1, pbar=True, update_every=16, verbose=True, weight_decay=1.0e-8)